In [5]:
from selenium import webdriver
import requests
import time
from sanitize_filename import sanitize
import os
import pickle

1. start the browser, go to linkedin learn and log into your account

In [8]:
driver = webdriver.Chrome()
driver.get('https://www.linkedin.com/learning/')

1a. save or retrieve cookie from a previous session

In [6]:
# save cookie
pickle.dump( driver.get_cookies() , open("cookies.pkl","wb"))

In [12]:
# retrieve saved cookie
cookies = pickle.load(open("cookies.pkl", "rb"))
for cookie in cookies:
    driver.add_cookie(cookie)

2. go to the the couse

In [40]:
link = 'https://www.linkedin.com/learning/bim-vdc-for-construction'

In [44]:
downloadAllVideos(driver, link)

BIM-VDC for Construction/0 - Bringing a project to life with VDC.mp4 successfully retrieved
BIM-VDC for Construction/1 - Winning your proposals.mp4 successfully retrieved
BIM-VDC for Construction/2 - Preconstruction.mp4 successfully retrieved
BIM-VDC for Construction/3 - Standard documentation.mp4 successfully retrieved
BIM-VDC for Construction/4 - Software and hardware.mp4 successfully retrieved
BIM-VDC for Construction/5 - Level of Development (LOD).mp4 successfully retrieved
BIM-VDC for Construction/6 - Authoring content.mp4 successfully retrieved
BIM-VDC for Construction/7 - Establishing BIM Plan and requirements.mp4 successfully retrieved
BIM-VDC for Construction/8 - BIM kickoff.mp4 successfully retrieved
BIM-VDC for Construction/9 - Coordinating models.mp4 successfully retrieved
BIM-VDC for Construction/10 - Bidding and negotiation.mp4 successfully retrieved
BIM-VDC for Construction/11 - 4D modeling.mp4 successfully retrieved
BIM-VDC for Construction/12 - Safety and risk assessme

In [41]:
def videoLinks(sidebar):
    collection = []
    # video links are in the side bar
    for i, e in enumerate(sidebar):
        link = e.find_element_by_tag_name('a').get_attribute('href')
        # the title of the video is the title of the class module
        name = e.find_element_by_class_name('classroom-toc-item__title').text
        # the quiz modules are skipped
        if link.find('quiz') > 0:
            pass
        else:
            # the [title, link] is added as list to the collection
            collection.append([str(i) + ' - ' + sanitize(name), link])
    # the [title, link] pairs are returned for the next step
    return collection

In [45]:
def downloadAllVideos(driver, link):
    driver.get(link)
    # get course title
    title = driver.find_element_by_tag_name('h1')
    title = sanitize(title.text)
    # make a directory with the course title if one does not exist yet
    if not os.path.isdir(title):
        os.mkdir(title)
    # get video sidebar
    sidebar = driver.find_elements_by_class_name('classroom-toc-item')
    
    pairs = videoLinks(sidebar)
    for p in pairs:
        time.sleep(5)
        downloadVideo(driver, p, title)
    print(title + 'complete')

In [43]:
def downloadVideo(driver, pair, folder):
    # destructure the variables we need
    [name, link] = pair
    # go to the video page
    driver.get(link)
    # wait 5 seconds the for the page to load
    # TODO: randomize
    time.sleep(5)
    # grab the video source
    videoFrame = driver.find_element_by_tag_name('video').get_attribute('src')
    # the video can be grabbed with a bare request
    s = requests.Session()
    response = s.get(videoFrame, stream=True)
    # 200 = good
    if response.status_code == 200:
        filename = folder + '/' + name +'.mp4'
        with open( filename, 'wb') as f:
            f.write(response.content)
        print(filename + ' successfully retrieved')
        